In [42]:
import pandas as pd

### ground truth

In [ ]:
# the ground truth values 
real_values = {
    'date': ['1/2024', '2/2024', '3/2024', '4/2024', '5/2024', '6/2024', '7/2024', '8/2024', '9/2024', '10/2024', '11/2024', '12/2024'],
    'real_consumo': [4885.10, 4880.14, 5391.35, 5925.34, 5295.28, 4996.13, 5256.78, 5069.64, 4694.26, 5215.3, 5273.88, 5501.23]
}

real_values = pd.DataFrame(real_values)
real_values = real_values.set_index('date')
real_values


### Xgboost with energy prediction

In [43]:
prediction_data_path = 'results/output_data/xgboost_with_energy.csv'
prediction_data = pd.read_csv(prediction_data_path)
prediction_data

,datetime,predicted_consumo
0,2024-01-01,5.965704
1,2024-01-02,5.853434
2,2024-01-03,5.578959
3,2024-01-04,5.149488
4,2024-01-05,5.155605
...,...,...
361,2024-12-27,5.780270
362,2024-12-28,5.700403
363,2024-12-29,5.885840
364,2024-12-30,5.737703


In [44]:
# we will transform all the predicted values in MWh, so multiply by 24
prediction_data['predicted_consumo'] = prediction_data['predicted_consumo'] * 24

# Now, we want to group by month and sum the predicted energy
prediction_data['month'] = pd.to_datetime(prediction_data['datetime']).dt.month
prediction_data['year'] = pd.to_datetime(prediction_data['datetime']).dt.year
prediction_data = prediction_data.drop(columns=['datetime'])
prediction_data = prediction_data.groupby(['month', 'year']).sum()

# Only for beauty we'll join the month and year columns
prediction_data['date'] = prediction_data.index
prediction_data['date'] = prediction_data['date'].apply(lambda x: f'{x[0]}/{x[1]}')
prediction_data = prediction_data.reset_index()
prediction_data = prediction_data.drop(columns=['month', 'year'])
prediction_data = prediction_data.set_index('date')

prediction_data

,predicted_consumo
date,
1/2024,4259.304041
2/2024,4534.367935
3/2024,4601.712974
4/2024,5617.293936
5/2024,5341.736280
6/2024,4121.975470
7/2024,4409.104291
8/2024,4827.393362
9/2024,4455.745277


In [49]:
# Now we will make a dataframe that contains the difference between the real values and the predicted values
diff = real_values 
diff['predicted_consumo'] = prediction_data['predicted_consumo']
diff['Erro'] = diff['real_consumo'] - diff['predicted_consumo']
diff['Erro %'] = diff['Erro'] / diff['real_consumo'] * 100

In [51]:
diff

,real_consumo,predicted_consumo,Erro,Erro %
date,,,,
1/2024,4885.10,4259.304041,625.795959,12.810300
2/2024,4880.14,4534.367935,345.772065,7.085290
3/2024,5391.35,4601.712974,789.637026,14.646369
4/2024,5925.34,5617.293936,308.046064,5.198791
5/2024,5295.28,5341.736280,-46.456280,-0.877315
6/2024,4996.13,4121.975470,874.154530,17.496633
7/2024,5256.78,4409.104291,847.675709,16.125379
8/2024,5069.64,4827.393362,242.246638,4.778379
9/2024,4694.26,4455.745277,238.514723,5.080987
